In [18]:
import yfinance as yf
import numpy as np 
import pandas as pd
import matplotlib
import mplfinance as mpf
import matplotlib.pylab as plt
import matplotlib.dates as mdates
import scipy.stats as stats
from datetime import datetime, timedelta
import math
import pickle
import gzip
%matplotlib inline

# Test PyTorch installation
import torch 
import torch.nn as nn
import pytorch_lightning as pl
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns

### Basic single-LSTM Layer RNN: 

Initializes device, enables GPU if available.

In [79]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU not available, CPU used


In [104]:
# Get data and define price
ticker = 'AAPL'
startDate = '2015-12-31'
endDate = '2016-12-31'

df = yf.Ticker(ticker).history(start=startDate, end=endDate)
del df['Dividends']
del df['Stock Splits']


In [105]:
price = df["Close"]
high_price = df["High"]
low_price = df["Low"]

In [106]:
# make scaler and normalize data
scaler = MinMaxScaler(feature_range=(-1,1))
data_normalized = scaler.fit_transform(price.values.reshape(-1,1))
data_normalized_high = scaler.fit_transform(high_price.values.reshape(-1,1))

In [121]:
data = np.concatenate((data_normalized,data_normalized_high), axis=1)

[-0.03901245]
[0.02217217]
[-0.03901245  0.02217217]


In [122]:
# put data into sliding window format
train_window = 20 # 20 days

def sliding_window(data, train_window):
    """
    Split Data into sliding window segemnts
    """
    data_window = []
    for i in range(len(data) - train_window):
        data_window.append(data[i: i+ train_window])
    return data_window
sliding_data = sliding_window(data, train_window)
# sliding_data_high = sliding_window(data_normalized_high, train_window)

In [123]:
# take sliding window and split it up into train/test data
def split_data(data):
    data = np.array(data) # convert to handle data nicer for splitting up training and testing
    train_size = int(0.7*len(data))
    test_size = int(0.3*len(data))
    # print(data[:train_size,:-1])
    
    x_train = data[:train_size,:-1] # training data and x data
    y_train = data[:train_size,-1] # training data and y label
    
    x_test = data[train_size:,:-1]
    y_test = data[train_size:,-1]
    
    x_train = torch.from_numpy(x_train).type(torch.Tensor)
    y_train = torch.from_numpy(y_train).type(torch.Tensor)
    x_test = torch.from_numpy(x_test).type(torch.Tensor)
    y_test = torch.from_numpy(y_test).type(torch.Tensor)
    return x_train, y_train, x_test, y_test

x_train, y_train, x_test, y_test = split_data(sliding_data)

In [133]:
class LSTM(nn.Module):
    ''' input_size: one price at a day
        hidden_layer_size: # of hidden layers along w number of neurons in each layer
        output_size: number of items in output (price at day)
    '''
    def __init__(self, input_layer_size = 2, hidden_layer_size = 10, num_layers = 3, output_layer_size = 1):
        super(LSTM, self).__init__()
        self.input_layer_size = input_layer_size
        self.hidden_layer_size = hidden_layer_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_layer_size, hidden_layer_size, num_layers,  batch_first = True)

        self.linear = nn.Linear(hidden_layer_size, output_layer_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_data):
        """
        h0 = inital hidden state, size of num of layers, a tenor of zeros with gradients
        c0 = inital cell state of lstm, size of num of layers, a tensor of zeros with gradients
        .requires_grad() = allows gradients and calculation of new gradients using backward()
        """
        hidden_state = torch.zeros(self.num_layers, input_data.size(0), self.hidden_layer_size).requires_grad_()# inital hidden state
        cell_state = torch.zeros(self.num_layers, input_data.size(0), self.hidden_layer_size).requires_grad_()# inital cell state
        lstm_out, (hidden, cell) = self.lstm(input_data, (hidden_state.detach(), cell_state.detach()))
        return self.linear(lstm_out[:, -1, :])
#         lstm_out, self.hidden_cell = self.lstm(input_data, self.hidden_cell)
#         predicitons = self.linear(lstm_out)
#         return predicitons[-1]
    
    

In [134]:
"""
Create model, loss function, optimization function
"""
model = LSTM(input_layer_size = 2, hidden_layer_size = 30, output_layer_size = 1,num_layers = 3)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [135]:
print(model)

LSTM(
  (lstm): LSTM(2, 30, num_layers=3, batch_first=True)
  (linear): Linear(in_features=30, out_features=1, bias=True)
)


In [136]:
epochs = 150
loss_arr = []

for i in range(epochs):
    # model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size), torch.zeros(1, 1, model.hidden_layer_size))
    y_pred = model(x_train)

    single_loss = loss_function(y_pred, y_train)
    loss_arr.append(single_loss.item())
    optimizer.zero_grad() # zero out gradients for new backprop 
    single_loss.backward()
    optimizer.step()

    #print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')
loss_arr = np.array(loss_arr)
#print(loss_arr)
print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

/Users/samghala/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:445: UserWarning:

Using a target size (torch.Size([163, 2])) that is different to the input size (torch.Size([163, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



epoch: 149 loss: 0.0073285704


In [128]:
predict = pd.DataFrame(scaler.inverse_transform(y_pred.detach().numpy()), columns=['Predict'])
original = pd.DataFrame(scaler.inverse_transform(y_train.detach().numpy()), columns=['Original'])

ValueError: Shape of passed values is (163, 2), indices imply (163, 1)

In [62]:
# Plot regular data and training data over epochs
# plt.figure(1)
# plt.plot(original.index, original.values, color = "blue", linewidth = 0.7)
# plt.plot(predict.index, predict.values, color = "red", linewidth = 0.7)
# plt.title("Stock vs Epochs")

# # Plot loss over epochs 
# plt.figure(2)
# plt.plot(loss_arr)
# plt.title("Loss")

# Interactive Plots
import plotly.express as px
import plotly.graph_objects as go
#print(original)
df = original
df1 = predict
new_df = pd.concat([df, df1], axis=1, sort=False)
#print(new_df)
loss_df = pd.DataFrame(loss_arr, columns=['Loss'])
fig = px.line(loss_df, y=loss_arr, title = "Loss")
fig.update_layout(title='Loss',
                   xaxis_title='Epoch',
                   yaxis_title='Loss')
fig.show()


In [63]:
import plotly.graph_objects as go

# Create random data with numpy
import numpy as np
np.random.seed(1)

new_original = []
new_pred = []
for i in range(len(original.values)):
    new_original.append(original.values[i][0])
for i in range(len(original.values)):
    new_pred.append(predict.values[i][0])
new_original = np.array(new_original)
new_pred = np.array(new_pred)

# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=original.index, y=new_original,
                    mode='lines',
                    name='Original'))
fig.add_trace(go.Scatter(x=original.index, y=new_pred,
                    mode='lines',
                    name='RNN'))
fig.update_layout(title='Original Data vs. RNN',
                   xaxis_title='Epoch',
                   yaxis_title='Stock')

fig.show()

In [44]:
from sklearn.metrics import mean_squared_error
import plotly.express as px
import plotly.graph_objects as go

# make predictions
y_test_pred = model(x_test)
#
print(type(y_train))
#
# invert predictions
y_train_pred = scaler.inverse_transform(y_pred.detach().numpy())
y_train = scaler.inverse_transform(y_train.detach().numpy())
y_test_pred = scaler.inverse_transform(y_test_pred.detach().numpy())
y_test = scaler.inverse_transform(y_test.detach().numpy())

<class 'torch.Tensor'>


In [45]:
model.eval()

for i in range(fut_pred):
    seq = torch.FloatTensor(test_inputs[-train_window:])
    with torch.no_grad():
        model.hidden = (torch.zeros(1, 1, model.hidden_layer_size),
                        torch.zeros(1, 1, model.hidden_layer_size))
        test_inputs.append(model(seq).item())

NameError: name 'fut_pred' is not defined

In [ ]:
actual_predictions = scaler.inverse_transform(np.array(test_inputs).reshape(-1, 1))

In [16]:
x = np.arange(split-train_window, split+train_window, 1)

fig, ax = plt.subplots(figsize=[20,12])
plt.plot(mid_prices)
plt.plot(x,actual_predictions, ls='--')

NameError: name 'split' is not defined

In [17]:
# shift train predictions for plotting

trainPredictPlot = np.empty_like(price)
trainPredictPlot[:] = np.nan
trainPredictPlot[train_window:len(y_train_pred)+train_window] = y_train_pred[0]

# shift test predictions for plotting
testPredictPlot = np.empty_like(price)
testPredictPlot[:] = np.nan
testPredictPlot[len(y_train_pred)+train_window-1:len(price)-1] = y_test_pred[0]

original = scaler.inverse_transform(price.values.reshape(-1,1))
print(testPredictPlot.shape)
print(trainPredictPlot.shape)
print(original)
predictions = np.append(trainPredictPlot, testPredictPlot, axis=0)
predictions = np.append(predictions, original, axis=0)
result = pd.DataFrame(predictions)

fig = go.Figure()
fig.add_trace(go.Scatter(go.Scatter(x=result.index, y=result[0],
                    mode='lines',
                    name='Train prediction')))
fig.add_trace(go.Scatter(x=result.index, y=result[1],
                    mode='lines',
                    name='Test prediction'))
fig.add_trace(go.Scatter(go.Scatter(x=result.index, y=result[2],
                    mode='lines',
                    name='Actual Value')))
fig.show()

(253,)
(253,)
[[104.53407547]
 [104.60267194]
 [102.5906333 ]
 [101.0587383 ]
 [ 97.81963848]
 [ 98.20835603]
 [ 99.40489965]
 [100.4947489 ]
 [ 98.53605112]
 [100.15939753]
 [ 98.33788636]
 [ 97.97970535]
 [ 98.0787814 ]
 [ 97.70533213]
 [101.60747208]
 [100.09844468]
 [100.51761017]
 [ 95.51036535]
 [ 96.02100757]
 [ 98.49794689]
 [ 97.80440818]
 [ 96.31824838]
 [ 97.74343636]
 [ 98.3334571 ]
 [ 96.35647283]
 [ 97.11508359]
 [ 97.09977103]
 [ 96.54805067]
 [ 96.11128206]
 [ 96.333504  ]
 [ 98.3641012 ]
 [ 99.49820502]
 [ 98.07292213]
 [ 97.90434478]
 [ 98.54802276]
 [ 96.86988016]
 [ 97.95032675]
 [ 98.45606514]
 [ 98.57099792]
 [ 98.40243955]
 [101.3449313 ]
 [101.513496  ]
 [102.08819153]
 [103.24526418]
 [102.3717333 ]
 [101.728049  ]
 [101.79701879]
 [101.83533182]
 [102.67056865]
 [102.86979011]
 [104.44831881]
 [105.51344018]
 [105.3831632 ]
 [105.47511449]
 [105.46747086]
 [106.08814204]
 [105.63604189]
 [105.28355564]
 [104.91572519]
 [106.82375232]
 [108.26436042]
 [107.8275

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)